# **model_XGBoost_2.0**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import joblib
from datetime import datetime

data = pd.read_csv('xrp_data_2024_new.csv')

# timestamp를 datetime으로 변환
data['timestamp'] = pd.to_datetime(data['timestamp'])

features = data.drop(columns=['timestamp', 'target_close'])
target = data['target_close']

# 시계열 데이터 분리
train_size = int(len(data) * 0.8)
X_train, X_test = features.iloc[:train_size], features.iloc[train_size:]
y_train, y_test = target.iloc[:train_size], target.iloc[train_size:]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, 'scaler_2.0.pkl')

model = xgb.XGBRegressor(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train_scaled, y_train)

joblib.dump(model, 'model_XGBoost_2.0.pkl')

['model_XGBoost_2.0.pkl']

# **model_XGBoost_3.0**

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import joblib
from datetime import datetime

# 데이터 로드 및 변환
data = pd.read_csv('xrp_data_2024_latest_2.csv') # 데이터 3056개
data['timestamp'] = pd.to_datetime(data['timestamp'])

# 피처와 타겟 분리
features = data.drop(columns=['timestamp', 'target_close'])
target = data['target_close']

# 시계열 데이터 분리
train_size = int(len(data) * 0.8)
X_train, X_test = features.iloc[:train_size], features.iloc[train_size:]
y_train, y_test = target.iloc[:train_size], target.iloc[train_size:]

# 가중치 부여 (최신 데이터에 높은 가중치)
train_indices = range(len(X_train))
weights = [1 + 2 * (i / len(train_indices)) for i in train_indices]

# 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
#joblib.dump(scaler, 'scaler_3.0.pkl')

# 모델 초기화
model = xgb.XGBRegressor(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# 모델 학습 (가중치 추가)
model.fit(X_train_scaled, y_train, sample_weight=weights)

# 모델 저장
#joblib.dump(model, 'model_XGBoost_3.0.pkl')
from sklearn.metrics import r2_score
y_pred = model.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2}")

R² Score: 0.9567573721722809
